In [1]:
import os
import torch
import pickle

from MeLU import MeLU
from options import config
from model_training import training
from data_generation import generate
from evidence_candidate import selection
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.nn import functional as F
import numpy as np

C:\Users\rnautiyal2\Anaconda3\envs\charanfyp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
master_path= "./ml"
if not os.path.exists("{}/".format(master_path)):
    os.mkdir("{}/".format(master_path))
    # preparing dataset. It needs about 22GB of your hard disk space.
    generate(master_path)

In [25]:
training_losses = []

In [26]:
testing_losses = []

In [24]:
epochs = [20, 40, 50, 60]

In [27]:
for epoch in epochs:
    # training model.
    melu = MeLU(config)
    state = "user_cold_state"
    model_filename = "{}/models_{}_{}.pkl".format(master_path,state,epoch)
    if not os.path.exists(model_filename):
        training_set_size = int(len(os.listdir("{}/{}".format(master_path,state))) / 4)
        supp_xs_s = []
        supp_ys_s = []
        query_xs_s = []
        query_ys_s = []
        for idx in tqdm(range(training_set_size)):
            supp_xs_s.append(pickle.load(open("{}/{}/supp_x_{}.pkl".format(master_path,state, idx), "rb")))
            supp_ys_s.append(pickle.load(open("{}/{}/supp_y_{}.pkl".format(master_path, state, idx), "rb")))
            query_xs_s.append(pickle.load(open("{}/{}/query_x_{}.pkl".format(master_path, state, idx), "rb")))
            query_ys_s.append(pickle.load(open("{}/{}/query_y_{}.pkl".format(master_path, state, idx), "rb")))
        total_dataset = list(zip(supp_xs_s, supp_ys_s, query_xs_s, query_ys_s))
        del(supp_xs_s, supp_ys_s, query_xs_s, query_ys_s)
        history = training(melu, total_dataset, batch_size=config['batch_size'], num_epoch=epoch, model_save=True, model_filename=model_filename)
        training_losses.append(history[-1])
    else:
        testing_set_size = int(len(os.listdir("{}/{}".format('testing',state))) / 4)
        supp_xs_s_testing = []
        supp_ys_s_testing = []
        query_xs_s_testing = []
        query_ys_s_testing = []
        for idx in tqdm(range(testing_set_size)):
            supp_xs_s_testing.append(pickle.load(open("{}/{}/supp_x_{}.pkl".format('testing',state, idx), "rb")))
            supp_ys_s_testing.append(pickle.load(open("{}/{}/supp_y_{}.pkl".format('testing', state, idx), "rb")))
            query_xs_s_testing.append(pickle.load(open("{}/{}/query_x_{}.pkl".format('testing', state, idx), "rb")))
            query_ys_s_testing.append(pickle.load(open("{}/{}/query_y_{}.pkl".format('testing', state, idx), "rb")))
        
        trained_state_dict = torch.load(model_filename)
        melu.load_state_dict(trained_state_dict)

        final_loss = []
        for i in range(len(supp_xs_s_testing)):
            prediction = melu.model(supp_xs_s_testing[i].cuda())
            temp_loss = F.mse_loss(prediction, supp_ys_s_testing[i].cuda().view(-1, 1))
            final_loss.append(temp_loss.item())

        for j in range(len(query_xs_s_testing)):
            prediction = melu.model(query_xs_s_testing[j].cuda())
            temp_loss = F.mse_loss(prediction, query_ys_s_testing[j].cuda().view(-1, 1))
            final_loss.append(temp_loss.item())
        testing_losses.append(np.mean(final_loss))
    



100%|██████████| 60/60 [04:44<00:00,  4.74s/it]


In [28]:
training_losses

[0.9896575985728083,
 0.8461494526347598,
 0.8154991994032988,
 0.7716594467291961]

In [34]:
for epoch in epochs:
    model_filename = "{}/models_{}_{}.pkl".format(master_path,state,epoch)
    testing_set_size = int(len(os.listdir("{}/{}".format('testing',state))) / 4)
    supp_xs_s_testing = []
    supp_ys_s_testing = []
    query_xs_s_testing = []
    query_ys_s_testing = []
    for idx in tqdm(range(testing_set_size)):
        supp_xs_s_testing.append(pickle.load(open("{}/{}/supp_x_{}.pkl".format('testing',state, idx), "rb")))
        supp_ys_s_testing.append(pickle.load(open("{}/{}/supp_y_{}.pkl".format('testing', state, idx), "rb")))
        query_xs_s_testing.append(pickle.load(open("{}/{}/query_x_{}.pkl".format('testing', state, idx), "rb")))
        query_ys_s_testing.append(pickle.load(open("{}/{}/query_y_{}.pkl".format('testing', state, idx), "rb")))

    trained_state_dict = torch.load(model_filename)
    melu.load_state_dict(trained_state_dict)

    final_loss = []
    for i in range(len(supp_xs_s_testing)):
        prediction = melu.model(supp_xs_s_testing[i].cuda())
        temp_loss = F.mse_loss(prediction, supp_ys_s_testing[i].cuda().view(-1, 1))
        final_loss.append(temp_loss.item())

    for j in range(len(query_xs_s_testing)):
        prediction = melu.model(query_xs_s_testing[j].cuda())
        temp_loss = F.mse_loss(prediction, query_ys_s_testing[j].cuda().view(-1, 1))
        final_loss.append(temp_loss.item())
    testing_losses.append(np.mean(final_loss))

100%|██████████| 67/67 [00:00<00:00, 325.24it/s]


In [44]:
testing_losses_1 = []
for epoch in tqdm(epochs):
    model_filename = "{}/models_{}_{}.pkl".format("models",state,epoch)
    trained_state_dict = torch.load(model_filename)
    melu.load_state_dict(trained_state_dict)

    final_loss = []
    for i in tqdm(range(testing_set_size)):
        prediction = melu.forward(supp_xs_s_testing[i].cuda(), supp_ys_s_testing[i].cuda(), query_xs_s_testing[i].cuda(), 5)
        temp_loss = F.mse_loss(prediction, query_ys_s_testing[i].cuda().view(-1, 1))
        final_loss.append(temp_loss.item())
    testing_losses_1.append(np.mean(final_loss))

100%|██████████| 4/4 [00:15<00:00,  3.77s/it]


In [45]:
testing_losses_1

[1.0591542207038225,
 1.0578843508638553,
 1.0598142247146636,
 1.0597989303407385]

In [39]:
supp_xs_s_testing[0]

tensor([[5, 0, 0,  ..., 0, 0, 0],
        [5, 0, 0,  ..., 0, 0, 0],
        [5, 0, 0,  ..., 0, 0, 0],
        ...,
        [5, 0, 0,  ..., 0, 0, 0],
        [5, 0, 0,  ..., 0, 0, 0],
        [5, 0, 0,  ..., 0, 0, 0]])

In [35]:
testing_losses

[1.0464284166471283,
 1.0464284166471283,
 1.0464284166471283,
 1.0464284166471283,
 1.0691450452849047,
 1.020958093120091,
 1.0298598736746987,
 1.0464284166471283]

In [41]:
training_losses

[0.9896575985728083,
 0.8461494526347598,
 0.8154991994032988,
 0.7716594467291961]

In [40]:
testing_losses[4:]

[1.0691450452849047, 1.020958093120091, 1.0298598736746987, 1.0464284166471283]

In [42]:
epochs

[20, 40, 50, 60]

In [43]:
state

'user_cold_state'

## Model 